**REPASO SESIÓN 7 - SQL-NOSQL**

In [ ]:
ALTER TABLE PRUEBAS_ALUMNOS.DBO.AFR_FACT_COSTE_DEPARTAMENTO
ADD MODIFICADO_SN BIT NOT NULL DEFAULT 0;
    ELIMINADO_SN BIT NOT NULL DEFAULT 0;

In [4]:
SELECT TOP 10 * FROM PRUEBAS_ALUMNOS.DBO.AFR_FACT_COSTE_DEPARTAMENTO

(0 rows affected)

Total execution time: 00:00:00.061

COMERCIAL_ID,MES_ID,DEPARTAMENTO_ID,DEPARTAMENTO_DESC,COSTE_DEPARTAMENTO,MODIFICADO_SN,ELIMINADO_SN


In [5]:
TRUNCATE TABLE PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO;

Commands completed successfully.

Total execution time: 00:00:00.048

In [10]:
USE PRUEBAS_ALUMNOS;
GO

CREATE OR ALTER PROCEDURE DBO.PR_AFR_REPARTO_COSTE_DPTO
    @p_mes_inicial INTEGER, --PARAMETRO 1
    @p_mes_final INTEGER --PARAMETRO 2

AS
BEGIN
    DECLARE @ORIGEN TABLE (
        COMERCIAL_ID VARCHAR(50) NOT NULL,
        MES_ID INT NOT NULL,
        DEPARTAMENTO_ID INT NOT NULL,
        DEPARTAMENTO_DESC VARCHAR(15),
        COSTE_DEPARTAMENTO DECIMAL(15,2)
    );

    WITH COMERCIAL_HORAS_MES AS (
        SELECT F.COMERCIAL_ID,
               F.MES_ID,
               SUM(F.CANTIDAD_HORAS) AS TOTAL_HORAS_COMERCIAL_MES
        FROM NSLDINA.DBO.FACT_COMERCIAL_HORAS F
        GROUP BY F.COMERCIAL_ID,
                 F.MES_ID
        ),

    PCT_COMERCIAL_DPTO_MES AS (
        SELECT F.COMERCIAL_ID,
               F.MES_ID,
               F.DEPARTAMENTO_ID,
               F.CANTIDAD_HORAS/CAST(TOTAL_HORAS_COMERCIAL_MES AS NUMERIC(5,2)) AS PCT_DEPARTAMENTO
        FROM NSLDINA.DBO.FACT_COMERCIAL_HORAS F 
            INNER JOIN COMERCIAL_HORAS_MES C 
                ON F.COMERCIAL_ID = C.COMERCIAL_ID AND C.MES_ID = F.MES_ID 
        )

    INSERT INTO @ORIGEN (
            COMERCIAL_ID,
            MES_ID,
            DEPARTAMENTO_ID,
            DEPARTAMENTO_DESC,
            COSTE_DEPARTAMENTO)
    
    SELECT P.COMERCIAL_ID,
           P.MES_ID,
           P.DEPARTAMENTO_ID,
           D.DEPARTAMENTO_DESC,
           ROUND(FN.IMPORTE_NOMINA * P.PCT_DEPARTAMENTO, 2) AS COSTE_DEPARTAMENTO 
    FROM NSLDINA.DBO.FACT_NOMINA FN 
        INNER JOIN PCT_COMERCIAL_DPTO_MES P 
            ON FN.EMPLEADO_ID = P.COMERCIAL_ID AND FN.MES_ID = P.MES_ID
        
        INNER JOIN NSLDINA.DBO.DIM_DEPARTAMENTO D 
            ON P.DEPARTAMENTO_ID = D.DEPARTAMENTO_ID 
    
    WHERE P.MES_ID BETWEEN @p_mes_inicial AND @p_mes_final;

    BEGIN TRY
        BEGIN TRAN;

        MERGE INTO PRUEBAS_ALUMNOS.DBO.AFR_FACT_COSTE_DEPARTAMENTO AS AFR 
            USING @ORIGEN AS O 
                ON O.COMERCIAL_ID = AFR.COMERCIAL_ID AND O.MES_ID = AFR.MES_ID AND O.DEPARTAMENTO_ID = AFR.DEPARTAMENTO_ID 
            WHEN MATCHED THEN
                UPDATE SET 
                    AFR.DEPARTAMENTO_ID = O.DEPARTAMENTO_ID,
                    AFR.COSTE_DEPARTAMENTO = O.DEPARTAMENTO_DESC,
                    AFR.MODIFICADO_SN = 1,
                    AFR.ELIMINADO_SN = 0 
            WHEN NOT MATCHED BY TARGET THEN
                INSERT 
                (
                    COMERCIAL_ID,
                    MES_ID,
                    DEPARTAMENTO_ID,
                    DEPARTAMENTO_DESC,
                    COSTE_DEPARTAMENTO                    
                )

                VALUES
                (
                    O.COMERCIAL_ID,
                    O.MES_ID,
                    O.DEPARTAMENTO_ID,
                    O.DEPARTAMENTO_DESC,
                    O.COSTE_DEPARTAMENTO                    
                )

            WHEN NOT MATCHED BY SOURCE THEN 
                UPDATE SET 
                    AFR.MODIFICADO_SN = 0,
                    AFR.ELIMINADO_SN = 1
            ;

        COMMIT TRAN;
    END TRY

    BEGIN CATCH
        ROLLBACK TRAN;
    END CATCH
END;

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.298

In [12]:
TRUNCATE TABLE PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO;

EXEC PRUEBAS_ALUMNOS.dbo.PR_AFR_REPARTO_COSTE_DPTO 201904, 201912; --DE ABRIL 19 A DIC 19
EXEC PRUEBAS_ALUMNOS.dbo.PR_AFR_REPARTO_COSTE_DPTO 201908, 202004; --DE AGOSTO 19 A ABRIL 20

SELECT DISTINCT MES_ID,
                MODIFICADO_SN,
                ELIMINADO_SN
FROM PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO
ORDER BY MES_ID;

(270 rows affected)

(270 rows affected)

(270 rows affected)

(0 rows affected)

(9 rows affected)

Total execution time: 00:00:00.232

MES_ID,MODIFICADO_SN,ELIMINADO_SN
201904,0,0
201905,0,0
201906,0,0
201907,0,0
201908,0,0
201909,0,0
201910,0,0
201911,0,0
201912,0,0


In [14]:
SELECT COUNT(*) AS CONTEO FROM PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO

(1 row affected)

Total execution time: 00:00:00.051

CONTEO
270


PIVOT / UNPIVOT

In [2]:
SELECT COUNT (*) CONTEO FROM PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO

(1 row affected)

Total execution time: 00:00:00.108

CONTEO
270


In [4]:
WITH CONSULTA_PIVOTAR AS (
    SELECT COMERCIAL_ID,
           MES_ID,
           DEPARTAMENTO_DESC,
           COSTE_DEPARTAMENTO
    FROM PRUEBAS_ALUMNOS.dbo.ICR_FACT_COSTE_DEPARTAMENTO
)

SELECT COMERCIAL_ID,
       MES_ID,
       VENTAS,
       MARKETING,
       RRHH,
       LOGISTICA,
       FINANCIERO
FROM CONSULTA_PIVOTAR

PIVOT (
    SUM(COSTE_DEPARTAMENTO)
    FOR DEPARTAMENTO_DESC IN (
        VENTAS,
        MARKETING,
        RRHH,
        LOGISTICA,
        FINANCIERO
    )
) AS PIVOTADA

(78 rows affected)

Total execution time: 00:00:00.071

COMERCIAL_ID,MES_ID,VENTAS,MARKETING,RRHH,LOGISTICA,FINANCIERO
COMERCIAL 115,201904,274.46,209.88,265.23,242.17,205.27
COMERCIAL 115,201905,275.31,179.55,265.73,292.07,184.34
COMERCIAL 115,201906,251.53,208.86,273.98,249.28,213.35
COMERCIAL 115,201907,289.10,193.53,250.87,274.76,188.75
COMERCIAL 115,201908,242.17,198.35,281.38,272.15,202.96
COMERCIAL 115,201909,190.43,276.99,220.11,225.06,284.41
COMERCIAL 115,201910,217.86,296.19,208.07,183.59,291.29
COMERCIAL 115,201911,213.03,266.28,207.95,235.85,273.89
COMERCIAL 115,201912,195.14,292.71,218.93,192.76,297.47
COMERCIAL 115,202001,223.67,292.49,204.01,189.26,287.57


In [5]:
SELECT TOP 10 * FROM NSLDINA.DBO.DIM_COMERCIAL

(6 rows affected)

Total execution time: 00:00:00.093

POSICIONABLE_ID,COMERCIAL_ID,SALARIO_BASE,COMISION_CLIENTE
115,COMERCIAL 115,900,12
116,COMERCIAL 116,1400,0
121,COMERCIAL 121,840,16
129,COMERCIAL 129,900,12
135,COMERCIAL 135,900,12
137,COMERCIAL 137,980,10


In [6]:
SELECT POSICIONABLE_ID,
       COMERCIAL_ID,
       CONCEPTO,
       IMPORTE 
FROM NSLDINA.DBO.DIM_COMERCIAL

UNPIVOT (
    IMPORTE
    FOR CONCEPTO IN (
        SALARIO_BASE,
        COMISION_CLIENTE
    )
) AS UNP;

(12 rows affected)

Total execution time: 00:00:00.087

POSICIONABLE_ID,COMERCIAL_ID,CONCEPTO,IMPORTE
115,COMERCIAL 115,SALARIO_BASE,900
115,COMERCIAL 115,COMISION_CLIENTE,12
116,COMERCIAL 116,SALARIO_BASE,1400
116,COMERCIAL 116,COMISION_CLIENTE,0
121,COMERCIAL 121,SALARIO_BASE,840
121,COMERCIAL 121,COMISION_CLIENTE,16
129,COMERCIAL 129,SALARIO_BASE,900
129,COMERCIAL 129,COMISION_CLIENTE,12
135,COMERCIAL 135,SALARIO_BASE,900
135,COMERCIAL 135,COMISION_CLIENTE,12
